# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'06-29-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'06-29-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-06-30 04:21:41,33.93911,67.709953,117158,4794,70016.0,42348.0,Afghanistan,300.958112,4.091910
1,NaN,NaN,NaN,Albania,2021-06-30 04:21:41,41.15330,20.168300,132514,2456,130002.0,56.0,Albania,4604.698033,1.853389
2,NaN,NaN,NaN,Algeria,2021-06-30 04:21:41,28.03390,1.659600,139229,3708,96815.0,38706.0,Algeria,317.504421,2.663238
3,NaN,NaN,NaN,Andorra,2021-06-30 04:21:41,42.50630,1.521800,13900,127,13717.0,56.0,Andorra,17990.034298,0.913669
4,NaN,NaN,NaN,Angola,2021-06-30 04:21:41,-11.20270,17.873900,38682,894,33079.0,4709.0,Angola,117.695140,2.311152


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21,6/26/21,6/27/21,6/28/21,6/29/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,98734,103902,105749,107957,109532,111592,111592,111592,115751,117158
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132490,132490,132496,132497,132499,132506,132509,132512,132513,132514
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,135821,136294,136679,137049,137403,137772,138113,138465,138840,139229


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21,6/26/21,6/27/21,6/28/21,6/29/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,3934,4215,4293,4366,4452,4519,4519,4519,4730,4794
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2454,2454,2455,2455,2455,2455,2456,2456,2456,2456
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3631,3641,3650,3660,3669,3678,3685,3693,3699,3708


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21,6/26/21,6/27/21,6/28/21,6/29/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,64401,65071,65565,66102,66799,67183,67483,68288,69205,70016
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,129918,129918,129941,129955,129966,129975,129982,129990,129994,130002
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,94571,94822,95084,95331,95599,95878,96108,96335,96572,96815


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21,6/26/21,6/27/21,6/28/21,6/29/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7241,7242,7242,7244,7244,7244,7244,7244,7247,7247
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,21868,21901,21901,21921,21921,21945,21945,21945,21985,21985
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2345,2345,2345,2344,2344,2344,2344,2344,2345,2345


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21,6/26/21,6/27/21,6/28/21,6/29/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,113,113,113,113,113,113,113,113,113,113
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,314,314,314,314,314,314,314,314,314,314
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,59,59,59,60,60,60,60,60,60,60


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
656,1001.0,Autauga,Alabama,US,2021-06-30 04:21:41,32.539527,-86.644082,7247,113,NaN,NaN,"Autauga, Alabama, US",12971.415275,1.559266
693,1075.0,Lamar,Alabama,US,2021-06-30 04:21:41,33.779950,-88.096680,1472,37,NaN,NaN,"Lamar, Alabama, US",10662.803332,2.513587
694,1077.0,Lauderdale,Alabama,US,2021-06-30 04:21:41,34.901719,-87.656247,9640,251,NaN,NaN,"Lauderdale, Alabama, US",10395.884783,2.603734


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,33652098,604474,0.0
India,30316897,397637,29366601.0
Brazil,18513305,515985,16277785.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,33652098,604474,0.0,33047624.0,2021-06-30 04:21:41,37.950547,-91.419547
India,30316897,397637,29366601.0,552659.0,2021-06-30 04:21:41,23.088275,81.806127
Brazil,18513305,515985,16277785.0,1719535.0,2021-06-30 04:21:41,-12.669522,-48.480493
France,5835885,111230,404987.0,5319668.0,2021-06-30 04:21:41,6.430808,-34.730285
Russia,5428961,132314,4926182.0,370465.0,2021-06-30 04:21:41,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3818303,63621,0.0
Texas,2997318,52308,0.0
Florida,2365464,37772,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3818303,63621,0.0,3754682.0,2021-06-30 04:21:41,37.843962,-120.728594
Texas,2997318,52308,0.0,2945010.0,2021-06-30 04:21:41,31.660643,-98.653069
Florida,2365464,37772,0.0,2327692.0,2021-06-30 04:21:41,28.940755,-82.700744
New York,2114686,53676,0.0,2061010.0,2021-06-30 04:21:41,42.544151,-75.474183
Illinois,1391480,25656,0.0,1365824.0,2021-06-30 04:21:41,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1249875,24496,0.0
Arizona,Maricopa,559688,10276,0.0
Illinois,Cook,556951,10490,0.0
Florida,Miami-Dade,508464,6472,0.0
Texas,Harris,403149,6577,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1249875,24496,0.0,1225379.0,2021-06-30 04:21:41,34.308284,-118.228241,6037.0
Arizona,Maricopa,559688,10276,0.0,549412.0,2021-06-30 04:21:41,33.348359,-112.491815,4013.0
Illinois,Cook,556951,10490,0.0,546461.0,2021-06-30 04:21:41,41.841448,-87.816588,17031.0
Florida,Miami-Dade,508464,6472,0.0,501992.0,2021-06-30 04:21:41,25.611236,-80.551706,12086.0
Texas,Harris,403149,6577,0.0,396572.0,2021-06-30 04:21:41,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-27,111592,132512,138465,13882,38556,1263,4405247,224797,30528,650261,...,4748644,365866,109223,4,269635,15740,313721,6908,148568,46442
2021-06-28,115751,132513,138840,13882,38613,1263,4423636,224851,30560,650324,...,4771367,366915,109692,4,269635,16136,313851,6909,149661,47284
2021-06-29,117158,132514,139229,13900,38682,1263,4447701,224967,30602,650353,...,4791628,368178,110190,4,270654,16507,313851,6918,152056,48533


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-27,4519,2456,3693,127,889,42,92568,4508,910,10700,...,128364,5494,729,1,3068,76,3559,1360,2022,1736
2021-06-28,4730,2456,3699,127,891,42,93142,4510,910,10701,...,128367,5524,731,1,3068,78,3561,1361,2091,1749
2021-06-29,4794,2456,3708,127,894,42,93668,4514,910,10700,...,128390,5558,733,1,3084,80,3561,1361,2138,1761


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-27,68288,129990,96335,13697,32791,1221,4027510,216558,23781,637424,...,15722,342494,105006,3,250528,6319,307618,4025,124582,37817
2021-06-28,69205,129994,96572,13697,32819,1221,4046308,216645,23788,637545,...,15729,344803,105780,3,250528,6519,307892,4032,126441,37949
2021-06-29,70016,130002,96815,13717,33079,1221,4062245,216718,23806,637613,...,15734,346857,106416,3,251587,6764,307892,4055,128312,38323


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21,6/26/21,6/27/21,6/28/21,6/29/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7241,7242,7242,7244,7244,7244,7244,7244,7247,7247
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1468,1469,1469,1472,1472,1472,1472,1472,1472,1472
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9603,9612,9612,9621,9621,9632,9632,9632,9640,9640
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3133,3134,3134,3135,3135,3135,3135,3135,3136,3136
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,16278,16287,16287,16301,16301,16317,16317,16317,16319,16319


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-27,549934,71035,0,893147,347254,3815751,556775,349120,109682,49,...,866738,2994389,413950,24392,3835,679828,450404,163804,677252,61951
2021-06-28,550451,71181,0,893560,348220,3817372,557004,349261,109700,49,...,866883,2995223,414204,24402,3839,679917,450930,163967,677396,62099
2021-06-29,550451,71187,0,894106,348699,3818303,557347,349301,109712,49,...,866841,2997318,414565,24406,3858,680065,451248,163992,677531,62195


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-06-27        7244   21945    2344  2686   6967    1247   2255   14761   
2021-06-28        7247   21985    2345  2687   6975    1249   2255   14766   
2021-06-29        7247   21985    2345  2687   6975    1249   2255   14766   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-27         3733     1874  ...    2902    680     3265      797   
2021-06-28         3734     1874  ...    2903    680     3268      797   
2021-06-29         3734     1874  ...    2904    681     3268      797   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-27           4729  3801  2315          0      926    656  
2021-06-28           4739  3801  2318          0      926    656  
2021-06-29           4757  3801  2319          0      926    657  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-27,11336,374,0,17913,5893,63589,6777,8275,1694,0,...,12550,52273,2363,256,30,11396,5902,2876,8099,740
2021-06-28,11338,374,0,17913,5897,63598,6776,8276,1694,0,...,12557,52279,2365,256,30,11402,5911,2878,8109,740
2021-06-29,11338,374,0,17930,5905,63621,6788,8276,1694,0,...,12564,52308,2368,256,30,11412,5920,2879,8126,747


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-06-27         113     314      60   64    139      42     71     330   
2021-06-28         113     314      60   64    139      42     71     330   
2021-06-29         113     314      60   64    139      42     71     330   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-27          124       45  ...      30     12       31        7   
2021-06-28          124       45  ...      30     12       31        7   
2021-06-29          124       45  ...      32     13       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-27             40    11    13          0       26      6  
2021-06-28             40    11    13          0       26      6  
2021-06-29             40    11    13          0       26      6  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-27,0.000000,0.000023,0.002549,0.000000,0.000727,0.0,0.002755,0.000307,0.000951,0.000106,...,0.003091,0.002760,0.004534,0.0,0.004792,0.027080,0.000641,0.000290,0.017373,0.009214
2021-06-28,0.037270,0.000008,0.002708,0.000000,0.001478,0.0,0.004174,0.000240,0.001048,0.000097,...,0.004785,0.002867,0.004294,0.0,0.000000,0.025159,0.000414,0.000145,0.007357,0.018130
2021-06-29,0.012155,0.000008,0.002802,0.001297,0.001787,0.0,0.005440,0.000516,0.001374,0.000045,...,0.004246,0.003442,0.004540,0.0,0.003779,0.022992,0.000000,0.001303,0.016003,0.026415


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-27,0.000000,0.0,0.002171,0.0,0.002255,0.0,0.002719,0.000444,0.0,0.000093,...,0.000086,0.003654,0.002751,0.0,0.004255,0.027027,0.000844,0.001473,0.027961,0.006377
2021-06-28,0.046692,0.0,0.001625,0.0,0.002250,0.0,0.006201,0.000444,0.0,0.000093,...,0.000023,0.005461,0.002743,0.0,0.000000,0.026316,0.000562,0.000735,0.034125,0.007488
2021-06-29,0.013531,0.0,0.002433,0.0,0.003367,0.0,0.005647,0.000887,0.0,-0.000093,...,0.000179,0.006155,0.002736,0.0,0.005215,0.025641,0.000000,0.000000,0.022477,0.006861


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-27,0.011929,0.000062,0.002362,0.00000,0.002906,0.0,0.004280,0.000319,0.000126,0.000292,...,0.000254,0.006293,0.005217,0.0,0.005979,0.029656,0.001243,0.001493,0.021457,0.001483
2021-06-28,0.013428,0.000031,0.002460,0.00000,0.000854,0.0,0.004667,0.000402,0.000294,0.000190,...,0.000445,0.006742,0.007371,0.0,0.000000,0.031651,0.000891,0.001739,0.014922,0.003490
2021-06-29,0.011719,0.000062,0.002516,0.00146,0.007922,0.0,0.003939,0.000337,0.000757,0.000107,...,0.000318,0.005957,0.006012,0.0,0.004227,0.037582,0.000000,0.005704,0.014797,0.009855


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-27,0.00000,0.000000,NaN,0.000488,0.000000,0.000226,0.000419,0.000000,0.000228,0.0,...,0.000000,0.000059,0.000682,0.000000,0.000000,-0.000341,0.000000,0.000000,0.000000,0.000000
2021-06-28,0.00094,0.002055,NaN,0.000462,0.002782,0.000425,0.000411,0.000404,0.000164,0.0,...,0.000167,0.000279,0.000614,0.000410,0.001043,0.000131,0.001168,0.000995,0.000213,0.002389
2021-06-29,0.00000,0.000084,NaN,0.000611,0.001376,0.000244,0.000616,0.000115,0.000109,0.0,...,-0.000048,0.000699,0.000872,0.000164,0.004949,0.000218,0.000705,0.000152,0.000199,0.001546


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-06-27      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2021-06-28      0.000414  0.001823  0.000427  0.000372  0.001148  0.001604   
2021-06-29      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

Province_State                                      ...   Wyoming            \
Admin2         Butler   Calhoun  Chambers Cherokee  ...      Park    Platte   
2021-06-27        0.0  0.000000  0.000000      0.0  ...  0.000000  0.000000   
2021-06-28        0.0  0.000339  0.000268      0.0  ...  0.000345  0.000000   
2021-06-29        0.0  0.000000  0.000000      0.0  ...  0.000344  0.001471   

Province_State                                                           \
Admin2          Sheridan Sublette Sweetwater Teton     Uinta Unassigned   
2021-06-27      0.000000      0.0   0.000000   0.0  0.000000        NaN   
2021-06-28      0.000919      0.0   0.002115   0.0  0.001296        NaN   
2021-06-29      0.000000      0.0   0.003798   0.0  0.000431        NaN   

Province_State                     
Admin2         Washakie    Weston  
2021-06-27          0.0  0.000000  
2021-06-28          0.0  0.000000  
2021-06-29          0.0  0.001524  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-27,0.000000,0.0,NaN,-0.000056,0.000000,0.000315,0.001034,0.000000,0.0,NaN,...,0.000000,0.000153,0.003397,0.0,0.0,-0.000526,0.000000,0.000000,0.000000,0.000000
2021-06-28,0.000176,0.0,NaN,0.000000,0.000679,0.000142,-0.000148,0.000121,0.0,NaN,...,0.000558,0.000115,0.000846,0.0,0.0,0.000527,0.001525,0.000695,0.001235,0.000000
2021-06-29,0.000000,0.0,NaN,0.000949,0.001357,0.000362,0.001771,0.000000,0.0,NaN,...,0.000557,0.000555,0.001268,0.0,0.0,0.000877,0.001523,0.000347,0.002096,0.009459


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-06-27         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2021-06-28         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2021-06-29         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ...   Wyoming                              \
Admin2         Chambers Cherokee  ...      Park    Platte Sheridan Sublette   
2021-06-27          0.0      0.0  ...  0.000000  0.000000      0.0      0.0   
2021-06-28          0.0      0.0  ...  0.000000  0.000000      0.0      0.0   
2021-06-29          0.0      0.0  ...  0.066667  0.083333      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-27            0.0   0.0   0.0        NaN      0.0    0.0  
2021-06-28            0.0   0.0   0.0        NaN      0.0    0.0  
2021-06-29            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-27,0.004646,0.000026,0.002564,0.000096,0.002648,7.425376e-11,0.003801,0.000365,0.001036,0.000156,...,0.003308,0.003433,0.004580,1.129377e-21,0.005018,0.031592,0.000539,0.000289,0.018934,0.013668
2021-06-28,0.020958,0.000017,0.002636,0.000048,0.002063,3.712688e-11,0.003988,0.000303,0.001042,0.000126,...,0.004046,0.003150,0.004437,5.646886e-22,0.002509,0.028375,0.000476,0.000217,0.013146,0.015899
2021-06-29,0.016557,0.000012,0.002719,0.000672,0.001925,1.856344e-11,0.004714,0.000409,0.001208,0.000086,...,0.004146,0.003296,0.004489,2.823443e-22,0.003144,0.025684,0.000238,0.000760,0.014574,0.021157


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-27,0.004540,0.000108,0.002185,1.147419e-19,0.003100,6.378736e-14,0.003785,0.000328,7.279917e-27,0.000136,...,0.000124,0.004664,0.002638,0.0,0.005027,0.019786,0.000472,0.001067,0.028760,0.005444
2021-06-28,0.025616,0.000054,0.001905,5.737095e-20,0.002675,3.189368e-14,0.004993,0.000386,3.639959e-27,0.000115,...,0.000074,0.005062,0.002691,0.0,0.002513,0.023051,0.000517,0.000901,0.031442,0.006466
2021-06-29,0.019573,0.000027,0.002169,2.868548e-20,0.003021,1.594684e-14,0.005320,0.000636,1.819979e-27,0.000011,...,0.000126,0.005609,0.002713,0.0,0.003864,0.024346,0.000258,0.000451,0.026960,0.006664


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-27,0.008977,0.000065,0.002486,0.000117,0.003688,2.815534e-06,0.004765,0.000341,0.000259,0.000283,...,0.000388,0.007176,0.004870,2.710505e-20,0.005510,0.029364,0.000928,0.002301,0.022243,0.002334
2021-06-28,0.011203,0.000048,0.002473,0.000059,0.002271,1.407767e-06,0.004716,0.000371,0.000276,0.000237,...,0.000417,0.006959,0.006121,1.355253e-20,0.002755,0.030507,0.000909,0.002020,0.018582,0.002912
2021-06-29,0.011461,0.000055,0.002495,0.000759,0.005097,7.038834e-07,0.004328,0.000354,0.000517,0.000172,...,0.000367,0.006458,0.006067,6.776264e-21,0.003491,0.034045,0.000455,0.003862,0.016690,0.006384


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-27,0.000150,0.000043,NaN,0.000535,0.000241,0.000244,0.000671,0.000081,0.000203,4.466673e-142,...,0.000036,0.000275,0.000774,0.000066,0.002135,0.000047,0.000531,0.000103,0.000035,0.000357
2021-06-28,0.000545,0.001049,NaN,0.000499,0.001512,0.000334,0.000541,0.000242,0.000184,2.233336e-142,...,0.000102,0.000277,0.000694,0.000238,0.001589,0.000089,0.000849,0.000549,0.000124,0.001373
2021-06-29,0.000273,0.000567,NaN,0.000555,0.001444,0.000289,0.000578,0.000178,0.000147,1.116668e-142,...,0.000027,0.000488,0.000783,0.000201,0.003269,0.000153,0.000777,0.000351,0.000162,0.001459


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-06-27      0.000011  0.000179 -0.000014  0.000013  0.000217  0.000139   
2021-06-28      0.000213  0.001001  0.000206  0.000192  0.000683  0.000871   
2021-06-29      0.000106  0.000500  0.000103  0.000096  0.000341  0.000436   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-06-27      0.000349  0.000311  0.000093 -0.000033  ...  0.000130   
2021-06-28      0.000174  0.000325  0.000180 -0.000016  ...  0.000237   
2021-06-29      0.000087  0.000162  0.000090 -0.000008  ...  0.000291   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-06-27      0.000427  0.000255 -0.000273   0.000436  0.000101  0.000546   
2021-06-28      0.000213  0.000587 -0.000137   0.001275  0.000050  0.000921   
2021-06-29      0.000842  0.000293 -0.000068   0.002537  0.000025  0.000676   

Province_State                                     
Admin2         Unassigned  Washakie        Weston  
2021-06-27           -1.0 -0.000118  1.468281e-09  
2021-06-28           -1.0 -0.000059  7.341404e-10  
2021-06-29           -1.0 -0.000029  7.621955e-04  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-27,0.000141,0.000084,NaN,0.000381,0.000136,0.000467,0.000777,0.000041,0.000265,NaN,...,0.000190,0.000311,0.002966,2.337510e-10,0.000013,0.000127,0.000466,0.000148,0.000286,0.000257
2021-06-28,0.000158,0.000042,NaN,0.000190,0.000408,0.000304,0.000315,0.000081,0.000132,NaN,...,0.000374,0.000213,0.001906,1.168755e-10,0.000006,0.000327,0.000996,0.000422,0.000761,0.000128
2021-06-29,0.000079,0.000021,NaN,0.000570,0.000882,0.000333,0.001043,0.000041,0.000066,NaN,...,0.000466,0.000384,0.001587,5.843774e-11,0.000003,0.000602,0.001259,0.000385,0.001428,0.004794


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                        \
Admin2               Autauga       Baldwin   Barbour          Bibb   
2021-06-27      4.862270e-09  3.126219e-06  0.000530  3.531631e-18   
2021-06-28      2.431135e-09  1.563109e-06  0.000265  1.765815e-18   
2021-06-29      1.215567e-09  7.815547e-07  0.000132  8.829077e-19   

Province_State                                                      \
Admin2                Blount       Bullock        Butler   Calhoun   
2021-06-27      4.169659e-16  1.386512e-15  3.431601e-12  0.000577   
2021-06-28      2.084830e-16  6.932558e-16  1.715800e-12  0.000289   
2021-06-29      1.042415e-16  3.466279e-16  8.579002e-13  0.000144   

Province_State                              ...       Wyoming            \
Admin2              Chambers      Cherokee  ...          Park    Platte   
2021-06-27      1.923477e-06  3.853997e-23  ...  2.176454e-31  0.002841   
2021-06-28      9.617385e-07  1.926999e-23  ...  1.088227e-31  0.001420   
2021-06-29      4.808692e-07  9.634993e-24  ...  3.333333e-02  0.042377   

Province_State                                                          \
Admin2              Sheridan      Sublette    Sweetwater         Teton   
2021-06-27      1.643795e-33  7.221337e-54  3.130008e-06  4.238553e-07   
2021-06-28      8.218973e-34  3.610669e-54  1.565004e-06  2.119276e-07   
2021-06-29      4.109487e-34  1.805334e-54  7.825020e-07  1.059638e-07   

Province_State                                                       
Admin2                 Uinta Unassigned      Washakie        Weston  
2021-06-27      2.312965e-18   0.752941  4.486738e-46  3.814697e-07  
2021-06-28      1.156482e-18   0.752941  2.243369e-46  1.907349e-07  
2021-06-29      5.782412e-19   0.752941  1.121685e-46  9.536743e-08  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
